In [36]:
import pandas as pd

# 1. EXTRAÇÃO DO ARQUIVO
df = pd.read_csv('saude_estetica.csv', sep=';')
df.columns = df.columns.str.strip() # Limpa os nomes das colunas

print("Tabela carregada com sucesso!")
display(df.head())

Tabela carregada com sucesso!


,ID,Nome,Procedimento,Dias_Desde_Ultima_visita,news
0,1,Mariana Oliveira,Botox,180,"Claro, Mariana!\n\n""Mariana, na minha visão co..."
1,2,Ricardo Santos,Limpeza de Pele,30,"Olá, Ricardo!\n\nA Limpeza de Pele vai muito a..."
2,3,Beatriz Costa,Preenchimento,365,"Olá, Beatriz!\n\nComo sua especialista em esté..."
3,4,Tatiane Mares,Drenagem Linfatica,45,"Olá, Tatiane Mares!\n\nA Drenagem Linfática va..."


In [37]:
import json

# Extraindo a lista de IDs reais da sua planilha
user_ids = df['ID'].tolist()

def get_user(id):
    # Busca o cliente no DataFrame pelo ID
    user_data = df[df['ID'] == id]
    return user_data.iloc[0].to_dict() if not user_data.empty else None

# Criando a lista de usuários
users = [user for id in user_ids if (user := get_user(id)) is not None]

print(f"Extraímos {len(users)} usuários para processamento.")
# Opcional: ver o primeiro usuário em formato JSON
print(json.dumps(users[0], indent=2, ensure_ascii=False))

Extraímos 4 usuários para processamento.
{
  "ID": 1,
  "Nome": "Mariana Oliveira",
  "Procedimento": "Botox",
  "Dias_Desde_Ultima_visita": 180,
  "news": "Claro, Mariana!\n\n\"Mariana, na minha visão como especialista, o Botox vai muito além da estética. É um gesto de autocuidado que se traduz em **saúde e bem-estar**. Proporciona relaxamento muscular, previne tensões e resgata a leveza de uma expressão mais descansada, elevando sua confiança e qualidade de vida.\""
}


In [34]:
import google.generativeai as genai

GOOGLE_API_KEY = "Chave_removida_por_Seguranca"
genai.configure(api_key=GOOGLE_API_KEY.strip())

try:
    available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
    print(f"Modelos encontrados na sua conta: {available_models}")

    if available_models:
        model_name = available_models[0]
        model = genai.GenerativeModel(model_name)

        response = model.generate_content("Oi")
        print(f"✅ SUCESSO! Conectado ao modelo: {model_name}")
        print(f"Resposta: {response.text}")
    else:
        print("❌ Nenhum modelo de geração de conteúdo encontrado.")
except Exception as e:
    print(f"❌ Erro ao listar ou conectar: {e}")

Modelos encontrados na sua conta: ['models/gemini-2.5-flash', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-exp-1206', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'models/gemini-flash-latest', 'models/gemini-flash-lite-latest', 'models/gemini-pro-latest', 'models/gemini-2.5-flash-lite', 'models/gemini-2.5-flash-image-preview', 'models/gemini-2.5-flash-image', 'models/gemini-2.5-flash-preview-09-2025', 'models/gemini-2.5-flash-lite-preview-09-2025', 'models/gemini-3-pro-preview', 'models/gemini-3-flash-preview', 'models/gemini-3

In [35]:

for user in users:
  nome = user['Nome']
  procedimento = user['Procedimento']

  print(f"Gerando mensagem para: {nome}...")

  prompt = f"Aja como uma especialista em estética. Crie uma mensagem curta destacando que o procedimento {procedimento} não é apenas beleza, mas sim cuidado com a saúde e bem-estar para {nome}."

  try:
    response = model.generate_content(prompt)
    user['news'] = response.text.strip()
  except Exception as e:
    user['news'] = f"Olá {nome}, seu procedimento de {procedimento} é essencial para sua saúde!"

print("Transformação concluída com o loop 'for'!")

Gerando mensagem para: Mariana Oliveira...
Gerando mensagem para: Ricardo Santos...
Gerando mensagem para: Beatriz Costa...
Gerando mensagem para: Tatiane Mares...
Transformação concluída com o loop 'for'!


In [38]:
import pandas as pd
# Transforma a lista de volta em tabela para visualização
df_resultado = pd.DataFrame(users)
display(df_resultado[['Nome', 'Procedimento', 'news']])

,Nome,Procedimento,news
0,Mariana Oliveira,Botox,"Claro, Mariana!\n\n""Mariana, na minha visão co..."
1,Ricardo Santos,Limpeza de Pele,"Olá, Ricardo!\n\nA Limpeza de Pele vai muito a..."
2,Beatriz Costa,Preenchimento,"Olá, Beatriz!\n\nComo sua especialista em esté..."
3,Tatiane Mares,Drenagem Linfatica,"Olá, Tatiane Mares!\n\nA Drenagem Linfática va..."


In [39]:
def update_user(users_list):
    # Transforma a lista de volta em DataFrame
    df_final = pd.DataFrame(users_list)

    # Salva o arquivo (Isso é o equivalente ao PUT na API)
    df_final.to_csv('saude_estetica.csv', index=False, sep=';', encoding='utf-8-sig')
    return True

# Chamando a função (Igual ele faz no loop ou ao final)
if update_user(users):
    print("Sucesso! Planilha 'atualizada' com as novas mensagens da IA.")

Sucesso! Planilha 'atualizada' com as novas mensagens da IA.


In [41]:
# ==========================================
# 3. CARREGAMENTO (LOAD)
# ==========================================

# No vídeo original, o instrutor utiliza uma API externa para salvar os dados (PUT).
# Para este projeto de estética, simulamos esse comportamento através da função 'update_user'.
# O nosso "Banco de Dados" aqui é o arquivo CSV final.

def update_user(users_list):
    """
    Simula o 'Update' do CRUD.
    Transforma a lista de dicionários (JSON) de volta em uma tabela (DataFrame)
    e persiste os dados em um novo arquivo CSV.
    """
    try:
        # Converte a lista atualizada (que agora contém a coluna 'news') em DataFrame
        df_final = pd.DataFrame(users_list)

        # Salva o arquivo final com codificação correta para acentos (UTF-8-SIG)
        df_final.to_csv('santander_dev_week_estetica_final.csv', index=False, sep=';', encoding='utf-8-sig')
        return True
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")
        return False

# Executando a carga dos dados
if update_user(users):
    print("✅ PROCESSO ETL CONCLUÍDO COM SUCESSO!")
    print("Arquivo 'santander_dev_week_estetica_final.csv' gerado.")

    # Exibe o resultado final para conferência rápida
    display(pd.DataFrame(users)[['Nome', 'Procedimento', 'news']].head())

✅ PROCESSO ETL CONCLUÍDO COM SUCESSO!
Arquivo 'santander_dev_week_estetica_final.csv' gerado.


,Nome,Procedimento,news
0,Mariana Oliveira,Botox,"Claro, Mariana!\n\n""Mariana, na minha visão co..."
1,Ricardo Santos,Limpeza de Pele,"Olá, Ricardo!\n\nA Limpeza de Pele vai muito a..."
2,Beatriz Costa,Preenchimento,"Olá, Beatriz!\n\nComo sua especialista em esté..."
3,Tatiane Mares,Drenagem Linfatica,"Olá, Tatiane Mares!\n\nA Drenagem Linfática va..."


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Nome').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Procedimento').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 10024 (\N{SPARKLES}) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('news').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Procedimento'].value_counts()
    for x_label, grp in _df_3.groupby('Nome')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Nome')
_ = plt.ylabel('Procedimento')

/usr/local/lib/python3.12/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 10024 (\N{SPARKLES}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 10024 (\N{SPARKLES}) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['news'].value_counts()
    for x_label, grp in _df_4.groupby('Procedimento')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Procedimento')
_ = plt.ylabel('news')